<a href="https://colab.research.google.com/github/stawiskm/QSAR_LightGBM/blob/main/MAO_B_Regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ACI Project: QSAR Regression models with RF and LightGBM on MAO-B inhibitors**
---


Marc Jermann   \ Patrick Meier


---

While quantitative structure-activity relationship
models are increasingly used nowadays, this study compares
two different machine learning methods in just such QSAR
models. The Light Gradient Boosting Machine method uses the
well established decision tree approach and has been optimized
for larger datasets. To test the applicability of this method, we
compare it with a Random Forest model, the existing state-of-the-
art method for such QSAR models, in three different applications.


---

In this Jupyter notebook, we will be building a real-life **data science project**.

---

## **Introduction**
Parkinson's disease is a progressive nervous system disorder that affects movement. Symptoms start gradually, sometimes starting with a barely noticeable tremor in just one hand. Tremors are common, but the disorder also commonly causes stiffness or slowing of movement.

In the early stages of Parkinson's disease, your face may show little or no expression. Your arms may not swing when you walk. Your speech may become soft or slurred. Parkinson's disease symptoms worsen as your condition progresses over time.

Although Parkinson's disease can't be cured, medications might significantly improve your symptoms. Occasionally, your doctor may suggest surgery to regulate certain regions of your brain and improve your symptoms.
(https://www.mayoclinic.org/diseases-conditions/parkinsons-disease/symptoms-causes/syc-20376055)

### *Monoamine Oxidase B*
Monoamine Oxidase Type B (MAO-B) is an enzyme in our body that breaks down several chemicals in the brain, including dopamine. By giving a medication that blocks the effect of MAO-B, an MAO-B inhibitor), more dopamine is available to be used by the brain. This can modestly improve many motor symptoms of PD. (https://www.parkinson.org/Understanding-Parkinsons/Treatment/Prescription-Medications/MAO-B-Inhibitors)

### *Literature regarding MAO B*

*  Comparative effectiveness of dopamine agonists and monoamine oxidase type-B inhibitors for Parkinson's disease: a multiple treatment comparison meta-analysis [(Eur J Clin Pharmacol)]()https://doi.org/10.1007/s00228-020-02961-6)

*   Monoamine Oxidase B Inhibitors in Parkinson's Disease [(CNS Neurol Disord Drug Targets)](https://doi.org/10.2174/1871527316666170124165222)

*  Long-term effectiveness of dopamine agonists and monoamine oxidase B inhibitors compared with levodopa as initial treatment for Parkinson's disease (PD MED): a large, open-label, pragmatic randomised trial [(Lancet)](https://doi.org/10.1016/s0140-6736(14)60683-8)









### *Monoamine oxidase type-B*

![picture](https://institute.progress.im/sites/default/files/styles/content_full/public/lundbeck_lic_illustrationer_02_09_depression_-_moa_of_mao-bi.png?itok=xCmdEPMO)

## **Investigating target protein**

In [1]:
# Install ChEMBL webresource client
! pip install chembl_webresource_client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 596 kB 49.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: itsdangerous
    Found existing installation: itsdangerous 1.1.0
    Uninstalling itsdangerous-1.1.0:
      Successfully uninstalled itsdangerous-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires itsdangerous<2.0,>=0.24, but you have itsdangerous 2.1.2 which is incompatible.


In [2]:
# Import necessary libraries
import pandas as pd 
from chembl_webresource_client.new_client import new_client

In [ ]:
# Target search for Monoamine oxidase B (MAO-B), Parkinson (~5 minutes)
target = new_client.target
target_query = target.search('Monoamine oxidase B')
targets = pd.DataFrame.from_dict(target_query)
targets.head(10)

In [ ]:
# Regex filter to search for the term Monoamine oxidase followed by a B
targets.loc[(targets["organism"] == "Homo sapiens") & (targets["pref_name"].str.contains('Monoamine oxidase.*B', regex=True))]

In [ ]:
# Selection of CHEMBL2039 as target, since it is referenced as Homo Sapiens and has the correct pref_name
selected_target = targets.target_chembl_id[4]
selected_target

In [ ]:
# Load activity data of the target filtered by half maximal inhibitory concentration (IC50) (~8min)
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")
df = pd.DataFrame.from_dict(res)
df.head()

In [ ]:
# Number of molecules (5067 Entries on April 25th 2022)
len(df)

In [ ]:
df.to_csv('MAOB_bioactivity.csv', index=False)

#### (possible shortcut)

In [ ]:
df = pd.read_csv('https://github.com/stawiskm/QSAR_LightGBM/raw/main/data/MAOB_bioactivity.csv')

In [ ]:
# Remove entries where standard_value is NAN
df2 = df[df.standard_value.notna()]
df2 = df2[df.canonical_smiles.notna()]
df2.head()

In [ ]:
# Subset of the dataframe only using the molecule_chembl_id, canonical_smiles and standard_value
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2[selection]
df3.head()

In [ ]:
# df.canonical_smiles.tolist()
test_smile = 'C=C(CNN)c1ccccc1.Cl'
test_smile
test_smile.split('.')

In [ ]:
def cleanSMILES(row):
    cpd = str(row.canonical_smiles).split('.')
    cpd_longest = max(cpd, key = len)
    return cpd_longest

In [ ]:
df3["smiles"] = df3.apply(cleanSMILES ,axis=1)
df3 = df3.drop("canonical_smiles",axis=1)
df3 = df3.rename({"smiles":"canonical_smiles"},axis=1)
df3.head()

In [ ]:
# Drop duplicate smiles
df3 = df3.drop_duplicates(['canonical_smiles'])
df3.head()

In [ ]:
# Number of unique molecules (4216 Entries on April 25th 2022)
len(df3.canonical_smiles.unique())

In [ ]:
df3.to_csv('MAOB_bioactivity_data_preprocessed.csv', index=False)

In [ ]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        #  Values greater than 100,000,000 will be fixed at 100,000,000 otherwise the negative logarithmic value will become negative.
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', 1)
        
    return x

In [ ]:
df3

In [ ]:
df_norm = norm_value(df3)
df_norm.head()

### Convert IC50 to pIC50

> To allow IC50 data to be more uniformly distributed, we will convert IC50 to the negative logarithmic scale which is essentially -log10(IC50).
This custom function pIC50() will accept a DataFrame as input and will:
Take the IC50 values from the standard_value column and converts it from nM to M by multiplying the value by 10 −9 
Take the molar value and apply -log10
Delete the standard_value column and create a new pIC50 column

In [ ]:
import numpy as np
# https://github.com/chaninlab/estrogen-receptor-alpha-qsar/blob/master/02_ER_alpha_RO5.ipynb

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', 1)
        
    return x

In [ ]:
df_final = pIC50(df_norm)
df_final

In [ ]:
df_final.to_csv('MAOB_bioactivity_final.csv')

## Feature encoding with PaDEL-Descriptor

A software that calculates molecular descriptors and fingerprints

In [ ]:
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.sh

## Data handling




In [ ]:
df = pd.read_csv('MAOB_bioactivity_final.csv')
df.head()

In [ ]:
selection = ['canonical_smiles','molecule_chembl_id']
df_sel = df[selection]
df_sel.head()

In [ ]:
df_sel.to_csv('molecule.smi', sep='\t', index=False, header=False)

In [ ]:
 ! cat molecule.smi | head -5

In [ ]:
! unzip padel.zip

In [ ]:
# Running the padel descriptor tool on the created molecule.smi data (~12min)
! bash padel.sh

In [ ]:
# Read in the file created from the PaDEL descriptor tool
df3_X = pd.read_csv('descriptors_output.csv')
df3_X.describe()

In [ ]:
# Create a target dataset containing the pIC50 value and the chembl id
df3_Y = df[['pIC50','molecule_chembl_id']]
df3_Y.head()

In [ ]:
# Merge the datasets together based on the unique ChEMBL id
df = pd.merge(df3_X,df3_Y, left_on=['Name'], right_on=['molecule_chembl_id'])
df.head()

In [ ]:
# Remove the ChEMBL id to create a dataframe only with the Pubchem features
df = df.drop(columns=['Name', 'molecule_chembl_id'])

In [ ]:
df.to_csv('MAOB_bioactivity_pubchem.csv', index=False)

#### (possible shortcut)

In [ ]:
df = pd.read_csv('https://github.com/stawiskm/QSAR_LightGBM/raw/main/data/MAOB_bioactivity_pubchem.csv')
df.head()

In [ ]:
# Remove all entries where pIC50 is NaN
df = df[df['pIC50'].notna()]

In [ ]:
len(df)

In [ ]:
# Create a feature dataset X without the pIC50 target value
X = df.drop('pIC50',axis=1)
X.head()

In [ ]:
# Create a target dataset containing only the pIC50 target 
y = df['pIC50']
y.head()

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [ ]:
# Split the data into 80% training and 20% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Check if the shape of the datasets is still correct
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

## **Building a Regression Model using Random Forest Classifier**

### **Random Forest Internal validation**

Then the Regression models are built, starting with the well-established Random Forest. 
First, the scoring is defined, which is used for all subsequent cross validations. Then, a stratified 5 fold cross-validation of a Random Forest model with optimized parameters is performed and the results of this internal validation are stored in a table.

In [ ]:
# The scorers can be either one of the predefined metric strings or a scorer
# callable, like the one returned by make_scorer

from sklearn.metrics import max_error,mean_absolute_error,r2_score,mean_squared_error,make_scorer

scoring = {"MaxError": make_scorer(max_error),
           "MAE": make_scorer(mean_absolute_error),
           "MSE": make_scorer(mean_squared_error),
           "R2": make_scorer(r2_score)}


In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
import time

rf = RandomForestRegressor(random_state=42)

parameters = {'max_depth': [5,10],
              'min_samples_leaf': [3,5],
              'min_samples_split': [5],
              'n_estimators': [1000]
              }

cv = KFold(n_splits=5,shuffle=True,random_state=42)

clf = GridSearchCV(rf, parameters,cv=cv,verbose=2,n_jobs=-1,scoring=scoring,refit="R2",return_train_score=True)

startTime = time.time()

#Model search
clf.fit(X_train, y_train)

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

In [ ]:
parameterlist = []
for parameter in list(parameters.keys()):
  feature = "param_"+parameter
  parameterlist.append(feature)
print(parameterlist)

In [ ]:
def defineSet(row):
    return str(row.variable).split("_")[1]
def defineMetric(row):
    return str(row.variable).split("_")[2]

In [ ]:
def defineRFModelName(df):
    name = "RF_model_"
    count = 1
    modelNames = []
    oldParam=[None]
    for param in df.params:
        if param not in oldParam:
            count = count + 1
            oldParam.append(param)
        modelNames.append(name+str(oldParam.index(param)))
    return modelNames

In [ ]:
dfGridsearch = pd.DataFrame(clf.cv_results_)
droplist = ['mean_fit_time', 'std_fit_time',
            'mean_score_time', 'std_score_time',
            'mean_test_MaxError', 'std_test_MaxError','rank_test_MaxError', 'mean_train_MaxError', 'std_train_MaxError',
            'mean_test_MAE', 'std_test_MAE','rank_test_MAE','mean_train_MAE', 'std_train_MAE', 
            'mean_test_MSE', 'std_test_MSE', 'rank_test_MSE', 'mean_train_MSE', 'std_train_MSE', 'split0_test_R2',
            'mean_test_R2', 'std_test_R2', 'rank_test_R2', 'mean_train_R2', 'std_train_R2'
            ]+parameterlist
dfScoreResults = dfGridsearch.drop(droplist,axis=1)
dfScoreResults = pd.melt(dfScoreResults, id_vars=['params'])
dfScoreResults["set"] = dfScoreResults.apply(defineSet, axis=1)
dfScoreResults["metric"] = dfScoreResults.apply(defineMetric, axis=1)
dfScoreResults = dfScoreResults.drop(["variable"],axis=1)
dfScoreResults["params"]=dfScoreResults["params"].astype(str)
dfScoreResults["method"]= "Random Forest"
dfScoreResults["model"] = defineRFModelName(dfScoreResults)

In [ ]:
searchparam = str(clf.best_params_)
dfScoreResults[dfScoreResults["params"]==searchparam].head(1)

In [ ]:
# for comparison
dfScoreRFresult = dfScoreResults[:]

### **Random Forest External validation**

This optimized Random Forest model is now trained again with the entire training set and the time needed to train the model is recorded. Finally, the test set is predicted for external validation and the scores of the various metrics are recorded in a table. 

In [ ]:
ModelResultsTable = pd.DataFrame()

In [ ]:
print(clf.best_params_)
model =  RandomForestRegressor(max_depth=clf.best_params_["max_depth"],
                               min_samples_leaf=clf.best_params_["min_samples_leaf"],
                               min_samples_split=clf.best_params_["min_samples_split"],
                               n_estimators=clf.best_params_["n_estimators"],
                               random_state=42)


startTime = time.time()

#Model training
model.fit(X_train, y_train)

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

y_pred = model.predict(X_test)
MaxError = max_error(y_test,y_pred)
MAE = mean_absolute_error(y_test,y_pred)
MSE = mean_squared_error(y_test,y_pred)
R2 = r2_score(y_test,y_pred)
output = pd.DataFrame({"Model":"RF_model_3","R2":R2,"MAE":MAE,"MSE":MSE,"MaxError":MaxError,"Params":[clf.best_params_],"ExecutionTime":executionTime})
ModelResultsTable = pd.concat([output, ModelResultsTable], ignore_index=True)
ModelResultsTable

## **Building a Regression Model using LGBMClassifier**

### **LightGBM Internal validation**

The next step is to create various LightGBM models using a gridsearch, combining previously defined parameters in all possible ways to create 48 different models. These models undergo the same stratified 5 fold cross validations with the same metrics as Random Forest models before. These results are also stored in a table for internal validation.

In [ ]:
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor(random_state=42)

parameters = {"boosting_type":["gbdt","dart"],
              "learning_rate":[0.063,0.126],
              "n_estimators" :[6,24,96],
              "num_leaves" : [32,64],
              "subsample_for_bin":[60000],
              'max_depth': [21,42]
              }
cv = KFold(n_splits=5, shuffle=True,random_state=42)

clf = GridSearchCV(lgbm, parameters,cv=cv,verbose=2,n_jobs=-1,scoring=scoring,refit="R2",return_train_score=True)

startTime = time.time()

#Model search
clf.fit(X_train, y_train)

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

In [ ]:
parameterlist = []
for parameter in list(parameters.keys()):
  feature = "param_"+parameter
  parameterlist.append(feature)
print(parameterlist)

In [ ]:
def defineModelName(df):
    name = "LGBM_model_"
    count = 1
    modelNames = []
    oldParam=[None]
    for param in df.params:
        if param not in oldParam:
            count = count + 1
            oldParam.append(param)
        modelNames.append(name+str(oldParam.index(param)))
    return modelNames

In [ ]:
dfGridsearch = pd.DataFrame(clf.cv_results_)
droplist = ['mean_fit_time', 'std_fit_time',
            'mean_score_time', 'std_score_time',
            'mean_test_MaxError', 'std_test_MaxError','rank_test_MaxError', 'mean_train_MaxError', 'std_train_MaxError',
            'mean_test_MAE', 'std_test_MAE','rank_test_MAE','mean_train_MAE', 'std_train_MAE', 
            'mean_test_MSE', 'std_test_MSE', 'rank_test_MSE', 'mean_train_MSE', 'std_train_MSE', 'split0_test_R2',
            'mean_test_R2', 'std_test_R2', 'rank_test_R2', 'mean_train_R2', 'std_train_R2'
            ]+parameterlist
dfScoreResults = dfGridsearch.drop(droplist,axis=1)
dfScoreResults = pd.melt(dfScoreResults, id_vars=['params'])
dfScoreResults["set"] = dfScoreResults.apply(defineSet, axis=1)
dfScoreResults["metric"] = dfScoreResults.apply(defineMetric, axis=1)
dfScoreResults = dfScoreResults.drop(["variable"],axis=1)
dfScoreResults["params"]=dfScoreResults["params"].astype(str)
dfScoreResults["method"]= "LightGBM"
dfScoreResults["model"] = defineModelName(dfScoreResults)
dfScoreResults = pd.concat([dfScoreResults,dfScoreRFresult],axis=0)

In [ ]:
modelsParamsDict = dfScoreResults[["model","params"]].set_index('model').drop_duplicates().to_dict('index')

In [ ]:
dfScoreResults

### **LightGBM External validation**

As with the Random Forest Model, the best four LightGBM models, the models that achieved the highest average accuracy values during the internal validation, are now trained again for the external validation and evaluated with the test set. 

LGBM_model_18

LGBM_model_24 

LGBM_model_23

LGBM_model_17


In [ ]:
import ast
clfmodel = ast.literal_eval(modelsParamsDict["LGBM_model_18"]["params"])
print(clfmodel)
model =  LGBMRegressor(boosting_type=clfmodel["boosting_type"],
                        learning_rate=clfmodel['learning_rate'],
                        n_estimators=clfmodel['n_estimators'],
                        num_leaves=clfmodel['num_leaves'],
                        subsample_for_bin=clfmodel["subsample_for_bin"],
                        max_depth=clfmodel["max_depth"],
                        random_state=42)
startTime = time.time()

#Model training
model.fit(X_train, y_train)

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

y_pred = model.predict(X_test)
MaxError = max_error(y_test,y_pred)
MAE = mean_absolute_error(y_test,y_pred)
MSE = mean_squared_error(y_test,y_pred)
R2 = r2_score(y_test,y_pred)

output = pd.DataFrame({"Model":"LGBM_model_18","R2":R2,"MAE":MAE,"MSE":MSE,"MaxError":MaxError,"Params":[clf.best_params_],"ExecutionTime":executionTime})
ModelResultsTable = pd.concat([output, ModelResultsTable], ignore_index=True)


In [ ]:
clfmodel = ast.literal_eval(modelsParamsDict["LGBM_model_24"]["params"])
print(clfmodel)
model =  LGBMRegressor(boosting_type=clfmodel["boosting_type"],
                        learning_rate=clfmodel['learning_rate'],
                        n_estimators=clfmodel['n_estimators'],
                        num_leaves=clfmodel['num_leaves'],
                        subsample_for_bin=clfmodel["subsample_for_bin"],
                        max_depth=clfmodel["max_depth"],
                        random_state=42)
startTime = time.time()

#Model training
model.fit(X_train, y_train)

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))


y_pred = model.predict(X_test)
MaxError = max_error(y_test,y_pred)
MAE = mean_absolute_error(y_test,y_pred)
MSE = mean_squared_error(y_test,y_pred)
R2 = r2_score(y_test,y_pred)

output = pd.DataFrame({"Model":"LGBM_model_24","R2":R2,"MAE":MAE,"MSE":MSE,"MaxError":MaxError,"Params":[clf.best_params_],"ExecutionTime":executionTime})
ModelResultsTable = pd.concat([output, ModelResultsTable], ignore_index=True)


In [ ]:
clfmodel = ast.literal_eval(modelsParamsDict["LGBM_model_23"]["params"])
print(clfmodel)
model =  LGBMRegressor(boosting_type=clfmodel["boosting_type"],
                        learning_rate=clfmodel['learning_rate'],
                        n_estimators=clfmodel['n_estimators'],
                        num_leaves=clfmodel['num_leaves'],
                        subsample_for_bin=clfmodel["subsample_for_bin"],
                        max_depth=clfmodel["max_depth"],
                        random_state=42)
startTime = time.time()

#Model training
model.fit(X_train, y_train)

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))


y_pred = model.predict(X_test)
MaxError = max_error(y_test,y_pred)
MAE = mean_absolute_error(y_test,y_pred)
MSE = mean_squared_error(y_test,y_pred)
R2 = r2_score(y_test,y_pred)

output = pd.DataFrame({"Model":"LGBM_model_23","R2":R2,"MAE":MAE,"MSE":MSE,"MaxError":MaxError,"Params":[clf.best_params_],"ExecutionTime":executionTime})
ModelResultsTable = pd.concat([output, ModelResultsTable], ignore_index=True)


In [ ]:
clfmodel = ast.literal_eval(modelsParamsDict["LGBM_model_17"]["params"])
print(clfmodel)
model =  LGBMRegressor(boosting_type=clfmodel["boosting_type"],
                        learning_rate=clfmodel['learning_rate'],
                        n_estimators=clfmodel['n_estimators'],
                        num_leaves=clfmodel['num_leaves'],
                        subsample_for_bin=clfmodel["subsample_for_bin"],
                        max_depth=clfmodel["max_depth"],
                        random_state=42)

startTime = time.time()

#Model training
model.fit(X_train, y_train)

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))


y_pred = model.predict(X_test)
MaxError = max_error(y_test,y_pred)
MAE = mean_absolute_error(y_test,y_pred)
MSE = mean_squared_error(y_test,y_pred)
R2 = r2_score(y_test,y_pred)

output = pd.DataFrame({"Model":"LGBM_model_17","R2":R2,"MAE":MAE,"MSE":MSE,"MaxError":MaxError,"Params":[clf.best_params_],"ExecutionTime":executionTime})
ModelResultsTable = pd.concat([output, ModelResultsTable], ignore_index=True)


## **Results**

### **Random Forest Internal validation**

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
sns.set_style("whitegrid")
sns.set(rc = {'figure.figsize':(4,8)})
ax = sns.boxplot(x="model", y="value", hue="set", data=dfScoreResults[(dfScoreResults["metric"]=="R2")&(dfScoreResults["method"]=="Random Forest")])
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
plt.plot()
plt.savefig('result_RF_TrainVsTest.png', dpi=300, format='png', bbox_inches='tight')

In [ ]:
sns.set_style("whitegrid")
sns.set(rc = {'figure.figsize':(4,8)})
ax = sns.boxplot(x="set", y="value", hue="metric", data=dfScoreResults[dfScoreResults["method"]=="Random Forest"])
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
plt.plot()
plt.savefig('result_RF_TrainVsTest.png', dpi=300, format='png', bbox_inches='tight')

### **LightGBM Internal validation**

In [ ]:
removeModels = dfScoreResults.loc[dfScoreResults["value"]<0]["model"].unique()

In [ ]:
removeModels = list(removeModels)
removeModels.append("RF_model_4")
removeModels.append("RF_model_1")
removeModels.append("RF_model_2")

In [ ]:
removeModels

In [ ]:
dfScoreResults= dfScoreResults.loc[~(dfScoreResults["model"].isin(removeModels))]

In [ ]:
print("\n Mean of R2:\n",dfScoreResults[(dfScoreResults["metric"]=="R2")&(dfScoreResults["set"]=="test")].groupby(["method"]).mean())
print("\n Std of R2:\n",dfScoreResults[(dfScoreResults["metric"]=="R2")&(dfScoreResults["set"]=="test")].groupby(["method"]).std())

In [ ]:
print("\n Mean of MSE:\n",dfScoreResults[(dfScoreResults["metric"]=="MSE")&(dfScoreResults["set"]=="test")].groupby(["method"]).mean())
print("\n Std of MSE:\n",dfScoreResults[(dfScoreResults["metric"]=="MSE")&(dfScoreResults["set"]=="test")].groupby(["method"]).std())

In [ ]:
print("\n Mean of MAE:\n",dfScoreResults[(dfScoreResults["metric"]=="MAE")&(dfScoreResults["set"]=="test")].groupby(["method"]).mean())
print("\n Std of MAE:\n",dfScoreResults[(dfScoreResults["metric"]=="MAE")&(dfScoreResults["set"]=="test")].groupby(["method"]).std())

In [ ]:
dfScoreResultsByParams = dfScoreResults[(dfScoreResults["metric"]=="R2")&(dfScoreResults["set"]=="test")].groupby(["model"]).mean()
dfScoreResultsByParams = dfScoreResultsByParams.sort_values('value', ascending = False)
dfScoreResultsByParamsSubset = dfScoreResultsByParams[dfScoreResultsByParams.value > 0.53]
print("\nMean R2 of best models:\n",dfScoreResultsByParamsSubset)
bestModels = list(dfScoreResultsByParamsSubset.index)
print("\n\nmodels showing best R2 in internal validation: \n",bestModels)
dfScoreResultsByParamsSTD = dfScoreResults[(dfScoreResults["metric"]=="R2")&(dfScoreResults["set"]=="test")].groupby(["model"]).std()
dfScoreResultsByParamsSTDSubset = dfScoreResultsByParamsSTD[dfScoreResultsByParams.value > 0.53]
print("\n Std R2 of best models:\n",dfScoreResultsByParamsSTDSubset)

In [ ]:
sns.set_style("whitegrid")
sns.set(rc = {'figure.figsize':(15,8)})
ax = sns.boxplot(x="model", y="value",hue="method", data=dfScoreResults[(dfScoreResults["metric"]=="R2")&(dfScoreResults["set"]=="test")],palette="Greys")
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
ax.set(title='R-squares of the models created during gridsearch')
ax.set_ylabel("R-squared")
ax.set_ylim(0.1,0.7)
plt.plot()
plt.savefig('result_R2_RF-Vs-LGBM.png', dpi=300, format='png', bbox_inches='tight')

In [ ]:
sns.set_style("whitegrid")
sns.set(rc = {'figure.figsize':(4,8)})
ax = sns.boxplot(x="model", y="value", hue="metric", data=dfScoreResults[(dfScoreResults["model"].isin(bestModels))&(dfScoreResults["set"]=="test")])
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
ax.set(title='Internal validation best models')
plt.plot()
plt.savefig('result_R2_RF-Vs-BestLGBM.png', dpi=300, format='png', bbox_inches='tight')

In [ ]:
sns.set_style("whitegrid")
sns.set(rc = {'figure.figsize':(8,4)})
ax = sns.boxplot(x="value", y="model", hue="metric", data=dfScoreResults[(dfScoreResults["model"].isin(bestModels))&(dfScoreResults["set"]=="test")&(dfScoreResults["metric"]!="MaxError")])
ax.set(title='Internal validation best models')
plt.plot()
plt.savefig('result_R2_RF-Vs-BestLGBM2.png', dpi=300, format='png', bbox_inches='tight')

In [ ]:
sns.set_style("whitegrid")
sns.set(rc = {'figure.figsize':(8,4)})
f, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, sharey=True)
ax = sns.boxplot(x="value", y="model", hue="metric", data=dfScoreResults[(dfScoreResults["model"].isin(["RF_model_3","LGBM_model_17"]))&(dfScoreResults["set"]=="test")&(dfScoreResults["metric"]!="MaxError")],ax=ax1)
ax = sns.boxplot(x="value", y="model", hue="metric", data=dfScoreResults[(dfScoreResults["model"].isin(["RF_model_3","LGBM_model_17"]))&(dfScoreResults["set"]=="test")&(dfScoreResults["metric"]=="MaxError")],ax=ax2)
f.suptitle('Internal validation best model')
ax2.set_ylabel("")
plt.subplots_adjust(wspace=.01, hspace=0)
plt.plot()
plt.savefig('result_R2_RF-Vs-BestLGBM3.png', dpi=300, format='png', bbox_inches='tight')

### **External validation**

In [ ]:
ModelResultsTable.to_csv('MAO-B_Results.csv', index=False)

In [ ]:
ModelResultsTable

In [ ]:
resultsTable = ModelResultsTable.drop(["Params"],axis=1)
resultsTable = resultsTable.set_index('Model')
sns.set(rc = {'figure.figsize':(8,4)})
ax = sns.heatmap(resultsTable, annot=True, fmt='.3g',linewidths=.5, cmap="seismic",vmin=.50,vmax=.95)
ax.set(title='External validation best models')
plt.savefig('result_RF-Vs-BestLGBM2.png', dpi=300, format='png', bbox_inches='tight', 
           dpi=300, format='png', bbox_inches='tight')

## **Conclusions**
The LightGBM is slowly finding its way into practical use and may replace the RF as the most popular classifier. Our study shows that the emerging LightGBM also has potential in QSAR model and can deliver equally good results as the popular Random Forest. Both methods delivered comparable scores, in the internal and external validation, predicting the MAO-B inhibitors of small molecules.



### **Zip files**

In [ ]:
! zip -r mao-b_results.zip . -i *.csv *.png